# Pulling Weekly and YTD Safety Data from Pro Football Reference and emailing the results

The following code will get a list of all safeties score each week and year-to-date then send an email to a group of safety enthusiasts

## Import libraries

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
from datetime import date

## Define a function to pull the data

In [ ]:
def safety_data (week, year):
  #define the base URLs to reference
  url = 'https://www.pro-football-reference.com/years/{}/week_{}.htm'.format(year, week)
  base_url= 'https://www.pro-football-reference.com/'
  
  #create a blank dataframe to hold the data
  weekly_safeties = pd.DataFrame(columns=['Tm', 'Detail','Week'])
  
  #open URL and pass to BeautifulSoup
  stats_page = BeautifulSoup(urlopen(url))
  
  #create a list of links from the page
  boxscore_links= stats_page.find_all('a')

  #filter only on links with boxscore and span in it
  boxscore_links = [x for x in boxscore_links if '"/boxscores/' in str(x) and 'span' in str(x)]
  
  #loop through the links
  for y in list(range(len(boxscore_links))):
    #pass each link to BeautifulSoup
    game_page = BeautifulSoup(urlopen(base_url + boxscore_links[y]['href']))
    
    #score table is call 'scoring'
    results = game_page.find(id="scoring")
    #read scores into dataframe
    df = pd.read_html(str(results))[0]
    df=df[['Tm', 'Detail']]
    #filter everything without 'safety'
    df=df[df["Detail"].str.contains("Safety")==True]
    #create a column to denote the week
    df['Week']=week
    weekly_safeties = pd.concat([weekly_safeties, df])
    #reset the index and add 1 to get a count
    weekly_safeties.reset_index(inplace=True, drop=True)
    weekly_safeties.index += 1
  return weekly_safeties


## Execute the code

In [ ]:
#calculate the week
#start date (which is kickoff) will need to be manually updated every year
start_date= date(2022, 9, 8)
today = date.today()
days = abs(start_date-today).days
nfl_week=days//7+1

#define the weeks to use and the year. 
if nfl_week == 1:
  week=[1]
else:
  week= list(range(1,nfl_week+1))
#Year should be updated manually every year
year=2022

#create a blank dataframe to hold the YTD numbers
ytd_safeties = pd.DataFrame(columns=['Tm', 'Detail','Week'])

#loop through the weeks and run the function defined above
for x in week:
  weekly_safeties=safety_data(x, year)
  #print(data)
  if len(weekly_safeties)>0:
    ytd_safeties = pd.concat([ytd_safeties, weekly_safeties])
ytd_safeties.reset_index(inplace=True, drop=True)
ytd_safeties.index += 1

weekly_safeties.rename(columns={'Tm': 'Team'}, inplace=True)
ytd_safeties.rename(columns={'Tm': 'Team'}, inplace=True)


## Send an Email

In [ ]:
pip install pretty_html_table

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from smtplib import SMTP
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from pretty_html_table import build_table

output_week= build_table(weekly_safeties, 'blue_light')
output_ytd= build_table(ytd_safeties, 'blue_light')

if len(weekly_safeties) == 0:
  msg1 = "There were no safeties this week<br><br>"
elif len(weekly_safeties) ==1:
  msg1= "Below is the safety scored in week {}:<br>".format(nfl_week) + output_week
else:
  msg1= "Below are the safeties scored in week {}:<br>".format(nfl_week) + output_week

if len(ytd_safeties) == 0:
  msg2="There were no safeties scored this year<br>"
elif len(ytd_safeties) == 1:
  msg2="Below is the only safety scored this year:<br>" + output_ytd
else:
  msg2="Below are the safeties scored year-to-date:<br>" + output_ytd

closer='<br>Brought to you by Safety First<br><br> If you want to unsubscribe, click <a href="https://www.mentalhealth.gov/get-help">here</a>.'

def send_mail(recipient, body):

    message = MIMEMultipart()
    message['Subject'] = 'Week {} Safeties!'.format(nfl_week)
    message['From'] = 'XXXXXXXX@gmail.com'
    message['To'] = recipient

    body_content = body
    message.attach(MIMEText(body_content, "html"))
    msg_body = message.as_string()

    server = SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(message['From'], 'PASSWORD')
    server.sendmail(message['From'], message['To'], msg_body)
    server.quit()

recipients=['XXXXXX@gmail.com','XXXXXX@gmail.com']

for recipient in recipients:
  send_mail(recipient, msg1 + msg2 + closer)